### MuMIDI Whole Song Level (w/ Basic Melody and Note Number indicator)

In [163]:
import os, sys
os.environ['PYTHONPATH'] = '/home/qihao/CS6207'
sys.path.append('/home/qihao/CS6207')
import miditoolkit
import numpy as np
import math
import pickle, glob
from tqdm import tqdm
from utils.indexed_datasets import IndexedDatasetBuilder
import multiprocessing as mp
import traceback

In [164]:
test_sample = "/home/qihao/CS6207/MelodyGLM/MDP/data/processed/wikifonia/7_dedup/wikifonia_1001_seg0_1_Seg1.mid"

In [165]:
def basic_melody (midi_pth: str):
    group_by_seg = {}
    basic_melody = {}
    midi = miditoolkit.MidiFile(midi_pth)
    assert len(midi.instruments) == 1  ## monophonic
    seg_len = midi.ticks_per_beat * 2
    
    for note in midi.instruments[0].notes:
        seg_id = note.start // seg_len
        if seg_id not in group_by_seg.keys():
            group_by_seg[seg_id] = []
        group_by_seg[seg_id].append(note)
    
    for seg_id, seg_notes in group_by_seg.items():
        freq = {}
        for note in seg_notes:
            if note.pitch not in freq.keys():
                freq[note.pitch] = 0
            freq[note.pitch] += 1
    
    return basic_melody

In [166]:
def stress (start, dur, reso=480):
    if dur in [reso, reso//2, reso//4, reso//8, reso//16, reso*2, reso*4]:
        ## categorise the note duration
        unit_len = 4 * dur
    else:
        unit_len = 4 * reso
    beat_pos = start - (start//unit_len) * unit_len
    beat_num = beat_pos // (unit_len//4)
    # beat_num = start % unit_len
    # print(f"dur:{dur}, pos:{beat_pos}, beat:{beat_num}, unit:{unit_len}")
    if beat_num == 0:
        return "<strong>"
    elif beat_num == 2:
        return "<substrong>"
    else:
        return "<weak>"

In [167]:
def prosody_avg_ver (midi_pth: str):
    ## use mean length to decide the overall length
    prosody = []
    
    midi = miditoolkit.MidiFile(midi_pth)
    ## group by bar:
    bar = {}
    ## calculate average note length
    note_durs = []
    strength, length = [], []
    reso = midi.ticks_per_beat
    for inst in midi.instruments:
        for i, note in enumerate(inst.notes):
            bar_num = np.floor(note.start/(8*reso))
            if bar_num not in bar.keys():
                bar[bar_num] = []
            bar[bar_num].append(note)
            dur = note.end - note.start
            note_durs.append(dur)
    ## calculate global average
    avg_dur = np.mean(note_durs)
    ## calculate bar level average
    bar_avg = {}
    for bar_num, bar_notes in bar.items():
        bar_avg[bar_num] = np.mean([note.end-note.start for note in bar_notes])
    for bar_num, bar_notes in bar.items():
        for note in bar_notes:
            strength = stress(start=note.start, dur=note.end-note.start, reso=reso)
            length = "<long>" if note.end-note.start>bar_avg[bar_num] else "<short>"
            # prosody.append(f"<{strength}, {length}>")
            prosody.append((strength, length))
    
    return prosody

In [168]:
def prosody (midi_pth: str):
    ## use absolute value
    prosody = []
    
    midi = miditoolkit.MidiFile(midi_pth)
    ## group by bar:
    bar = {}
    ## calculate average note length
    note_durs = []
    strength, length = [], []
    reso = midi.ticks_per_beat
    for inst in midi.instruments:
        for i, note in enumerate(inst.notes):
            strength = stress(start=note.start, dur=note.end-note.start, reso=reso)
            length = "<long>" if note.end-note.start>reso else "<short>"
            prosody.append((strength, length))
    
    return prosody

In [169]:
def basic_melody (midi_pth: str):
    prsd = prosody(midi_pth)
    midi = miditoolkit.MidiFile(midi_pth)
    notes = midi.instruments[0].notes
    assert len(prsd) == len(notes)
    
    group_by_beat2 = {}
    beat2 = midi.ticks_per_beat * 2
    for idx, note in enumerate(notes):
        beat2_id = note.start // beat2
        if beat2_id not in group_by_beat2.keys():
            group_by_beat2[beat2_id] = []
        group_by_beat2[beat2_id].append((idx, note))
    
    
    ## calculate the frequency
    group_by_freq = {}
    for beat2_id, beat2_notes in group_by_beat2.items():
        if beat2_id not in group_by_freq.keys():
            group_by_freq[beat2_id] = {}
        for note_id, note in beat2_notes:
            if note.pitch not in group_by_freq[beat2_id].keys():
                group_by_freq[beat2_id][note.pitch] = []
            group_by_freq[beat2_id][note.pitch].append((note_id, note))
    
    ## sort
    selected_pitch = {}
    for beat2_id, beat2_notes in group_by_freq.items():
        sorted_freq = sorted(beat2_notes.items(), key=lambda x:(len(x[1]), max([t[1].end-t[1].start for t in x[1]]), x[0]))
        if beat2_id not in selected_pitch.keys():
            selected_pitch[beat2_id] = []
        selected_pitch[beat2_id].append(sorted_freq[-1][0])
        selected_pitch[beat2_id].append(group_by_beat2[beat2_id].copy())
    
    return selected_pitch  

In [170]:
bm = basic_melody('/home/qihao/CS6207/dataset/test/一眼万年_seg0_1_Seg2.mid')
bm

{0: [76,
  [(0, Note(start=0, end=120, pitch=76, velocity=90)),
   (1, Note(start=240, end=360, pitch=76, velocity=90)),
   (2, Note(start=360, end=480, pitch=74, velocity=90)),
   (3, Note(start=480, end=600, pitch=76, velocity=90)),
   (4, Note(start=720, end=840, pitch=76, velocity=90)),
   (5, Note(start=840, end=960, pitch=74, velocity=90))]],
 1: [76,
  [(6, Note(start=960, end=1440, pitch=76, velocity=90)),
   (7, Note(start=1680, end=1800, pitch=69, velocity=90)),
   (8, Note(start=1800, end=1920, pitch=76, velocity=90))]],
 2: [74,
  [(9, Note(start=1920, end=2040, pitch=74, velocity=90)),
   (10, Note(start=2040, end=2160, pitch=74, velocity=90)),
   (11, Note(start=2160, end=2280, pitch=74, velocity=90)),
   (12, Note(start=2280, end=2400, pitch=74, velocity=90)),
   (13, Note(start=2400, end=2520, pitch=74, velocity=90)),
   (14, Note(start=2640, end=2760, pitch=71, velocity=90)),
   (15, Note(start=2760, end=3000, pitch=72, velocity=90))]],
 3: [69, [(16, Note(start=3000, 

In [171]:
print(len(prosody(test_sample)))

278


In [172]:
def phrasing (midi_pth: str):
    midi = miditoolkit.MidiFile(midi_pth)
    assert len(midi.instruments) == 1  ## monophonic
    reso = midi.ticks_per_beat
    notes = midi.instruments[0].notes.copy()
    
    long = []
    pause = []
    note_info = []
    
    for idx, note in enumerate(notes):
        note_bar = int(np.floor(note.start / (4 * reso))) ## a bar == 4 beat == 4 * 480 ticks
        note_pos = (note.start - (note_bar * 4 * reso)) ## relative position in the current bar
        note_pitch = note.pitch
        note_dur = note.end - note.start
        note_info.append((note_bar, note_pos, note_pitch, note_dur))
        if note_dur > reso:
            long.append(idx)
        if (idx > 0) and (notes[idx].start-notes[idx-1].end >= reso//2):
            pause.append(idx-1)
    
    union = list(set(long + pause))
    if 0 in union:
        union.remove(0)
    if len(notes)-1 in union:
        union.remove(len(notes)-1)
    union.sort()
    
    def dur(note: miditoolkit.Note):
        return abs(note.end-note.start)
    
    i = 1
    while i<len(union):
        if abs(union[i-1]-union[i]) == 1:
            if abs(dur(notes[union[i-1]])-dur(notes[union[i]])) > 240:
                union.remove(union[i])
            else:
                union.remove(union[i-1])
        i = i + 1
    
    ### annotate
    midi.markers=[]
    for k, b in enumerate(union):
        midi.markers.append(miditoolkit.Marker(time=notes[b].end, text=f"Phrase_{k}"))
    
    # midi.dump(os.path.join('./', os.path.basename(midi_pth)[:-4]+'_phrased.mid'))
    
    is_boundary = []
    for i in range(len(notes)):
        if i in union:
            is_boundary.append("<true>")
        else:
            is_boundary.append("<false>")
    
    assert len(note_info) == len(is_boundary)
    return is_boundary, note_info, union

In [173]:
def get_notes (midi_pth: str):
    midi = miditoolkit.MidiFile(midi_pth)
    assert len(midi.instruments) == 1  ## monophonic
    reso = midi.ticks_per_beat
    notes = midi.instruments[0].notes.copy()
    
    note_info = []
    
    for note in notes:
        note_info.append()

In [174]:
bound, note_info, _ = phrasing(test_sample)
print(len(bound), len(note_info))

278 278


In [175]:
def tokenise (midi_pth, event2word_dict):
    prsd = prosody(midi_pth)
    b_melody = basic_melody(midi_pth)
    bound, notes, _ = phrasing(midi_pth)
    assert len(prsd) == len(bound)
    src_words, tgt_words = [], []
    global_bar_id = 0
    last_item_bar = 0
    last_item_pos = -1
    
    ## bos
    src_words.append({
        'strength':event2word_dict['Prosody']['<s>'],
        'length':0,
        'phrase':0,
    })
    
    tgt_words.append({
        'bar': 0,
        'pos': 0,
        'token':event2word_dict['Token'][f"<s>"],
        'dur': 0,
        'phrase': 0,
    })
    
    ## source words
    src_sent_id = 0
    src_words.append({
        'strength':event2word_dict['Prosody'][f"<sent>"],
        'length':0,
        'phrase':0,
    })
    src_sent_id += 1
    for beat2_id, beat2_info in b_melody.items():
        b_pitch = beat2_info[0]
        b_notes = beat2_info[1]
        """
        src_words.append({
            'strength':event2word_dict['Prosody'][f"<note_{len(b_notes)}>"],
            'length':0,
            'phrase':0,
        })
        """
        src_words.append({
            'strength':event2word_dict['Prosody'][f"Pitch_{b_pitch}"],
            'length':0,
            'phrase':0,
        })
        for note_id, note in b_notes:
            strength, length = prsd[note_id][0], prsd[note_id][1]
            src_words.append({
                'strength':event2word_dict['Prosody'][f"{strength}"],
                'length':0,
                'phrase':0,
            })
            src_words.append({
                'strength':event2word_dict['Prosody'][f"{length}"],
                'length':0,
                'phrase':0,
            })
            if bound[note_id] == '<true>':
                src_words.append({
                    'strength':event2word_dict['Prosody'][f'<sent>'],
                    'length':0,
                    'phrase':0,
                })
                src_sent_id += 1

    
    src_words.append({
        'strength':event2word_dict['Prosody']['</s>'],
        'length':0,
        'phrase':0,
    })
    '''
    ## compound word version
    src_words.append({
        'strength':event2word_dict['Prosody'][prosody_word],
        'length':0,
        'phrase':0,
    })
    '''
    
    tgt_sent_id = 0
    for idx in range(len(notes)):
        if idx == 0:
            tgt_words.append({
                'bar':0,
                'pos':0,
                'token':event2word_dict['Token'][f"<sent>"],
                'dur':0,
                'phrase':0,
            })
            tgt_sent_id += 1
        note_bar = notes[idx][0]
        while global_bar_id <= note_bar:
            global_bar_id += 1
            tgt_words.append({
                'bar':0,
                'pos':0,
                'token':event2word_dict['Token'][f"Bar"],
                'dur':0,
                'phrase':0,
            })
            last_item_pos = 0
        
        note_pos = notes[idx][1]
        if global_bar_id == last_item_bar and last_item_pos == note_pos:
            continue
        tgt_words.append({
            'bar':0,
            'pos':0,
            'token':event2word_dict['Token'][f"Pos_{note_pos}"],
            'dur':0,
            'phrase':0,
        })
        last_item_pos = note_pos
        last_item_bar = global_bar_id
        
        
        tgt_words.append({
            'bar':0,
            'pos':0,
            'token':event2word_dict['Token'][f"Pitch_{notes[idx][2]}"],
            'dur':0,
            'phrase':0,
        })
        tgt_words.append({
            'bar':0,
            'pos':0,
            'token':event2word_dict['Token'][f"Dur_{notes[idx][3]}"],
            'dur':0,
            'phrase':0,
        })
        
        ## sentence bound
        if bound[idx] == '<true>':
            tgt_words.append({
                'bar':0,
                'pos':0,
                'token':event2word_dict['Token'][f"<sent>"],
                'dur':0,
                'phrase':0,
            })
            tgt_sent_id += 1
        
        '''
        tgt_words.append({
            'bar':event2word_dict['Bar'][f"Bar_{notes[idx][0]}"],
            'pos':event2word_dict['Pos'][f"Pos_{notes[idx][1]}"],
            'token':event2word_dict['Pitch'][f"Pitch_{notes[idx][2]}"],
            'dur':event2word_dict['Dur'][f"Dur_{notes[idx][3]}"],
            'phrase':event2word_dict['Phrase'][bound[idx]],
        })
        '''
    
    ## eos
    tgt_words.append({
        'bar':event2word_dict['Token'][f"</s>"],
        'pos':event2word_dict['Token'][f"</s>"],
        'token':event2word_dict['Token'][f"</s>"],
        'dur':event2word_dict['Token'][f"</s>"],
        'phrase':event2word_dict['Token'][f"</s>"],
    })
    
    return src_words, tgt_words

In [176]:
def data_to_binary (midi_pth, i, event2word_dict, split):
    try:
        src_words, tgt_words = tokenise(midi_pth, event2word_dict)
        if len(src_words) == 0 or len(tgt_words) == 0 or len(tgt_words) > 1024:
            return None
        
        data_sample = {
            'input_path': midi_pth,
            'item_name': os.path.basename(midi_pth),
            'src_words': src_words,
            'tgt_words': tgt_words,
            'word_length': len(tgt_words)
        }
        
        return [data_sample]
    
    except Exception as e:
        traceback.print_exc()
        return None

In [177]:
def data2binary(dataset_dirs, words_dir, split, word2event_dict, event2word_dict):
    # make dir
    save_dir = f'{words_dir}/{split}'
    os.makedirs(save_dir, exist_ok=True)
    
    midi_files = []
    for dataset_dir in dataset_dirs:
        midi_files.extend(glob.glob(os.path.join(os.path.join(dataset_dir, split), "*.mid")))
    
    futures = []
    ds_builder = IndexedDatasetBuilder(save_dir)  # index dataset
    p = mp.Pool(int(os.getenv('N_PROC', 2)))  # 不要开太大，容易内存溢出
    
    for i in range (len(midi_files)):
        futures.append(p.apply_async(data_to_binary, args=[midi_files[i], i, event2word_dict, split]))
    p.close()

    words_length = []
    all_words = []
    for f in tqdm(futures):
        item = f.get()
        if item is None:
            continue
        for i in range(len(item)):
            sample = item[i]
            words_length.append(sample['word_length'])
            all_words.append(sample)
            ds_builder.add_item(sample) # add item index

    # save 
    ds_builder.finalize()
    np.save(f'{words_dir}/{split}_words_length.npy', words_length)
    np.save(f'{words_dir}/{split}_words.npy', all_words)
    p.join()
    print(f'| # {split}_tokens: {sum(words_length)}; {split}_items: {len(words_length)}')
    
    return all_words, words_length

In [178]:
## shuffle and split dataset
def split_data(output_dir='/home/qihao/CS6207/dataset'):
    dataset_dirs = [
        '/home/qihao/CS6207/MelodyGLM/MDP/data/processed/pop909/7_dedup',
        '/home/qihao/CS6207/MelodyGLM/MDP/data/processed/wikifonia/7_dedup',
        '/home/qihao/CS6207/MelodyGLM/MDP/data/processed/mtc/7_dedup',
        '/home/qihao/CS6207/MelodyGLM/MDP/data/processed/sessions/7_dedup'
    ]
    all_files = []
    for dataset_dir in dataset_dirs:
        all_files.extend(glob.glob(os.path.join(dataset_dir, '*.mid')))
    ## shuffle
    print(f"|>>> Total Files: {len(all_files)}")
    
    indices = [i for i in range(len(all_files))]
    import random, shutil
    random.shuffle(indices)
    train_end = int(np.floor(0.8*len(all_files)))
    valid_end = int(train_end + np.floor(0.1*len(all_files)))
    train_idx = indices[:train_end]
    valid_idx = indices[train_end:valid_end]
    test_idx = indices[valid_end:]
    assert len(all_files) == len(train_idx)+len(valid_idx)+len(test_idx)
    print(f"|>>>>> Train Files: {len(train_idx)}")
    print(f"|>>>>> Valid Files: {len(valid_idx)}")
    print(f"|>>>>> Test Files: {len(test_idx)}")
    
    for split in ['train', 'test', 'valid']:
        os.makedirs(os.path.join(output_dir, split), exist_ok=True)
    
    for t in train_idx:
        shutil.copy(all_files[t], os.path.join(f'{output_dir}/train', os.path.basename(all_files[t])))
    for v in valid_idx:
        shutil.copy(all_files[v], os.path.join(f'{output_dir}/valid', os.path.basename(all_files[v])))
    for t in test_idx:
        shutil.copy(all_files[t], os.path.join(f'{output_dir}/test', os.path.basename(all_files[t])))

In [179]:
# split_data()

In [180]:
dataset_dirs = ['/home/qihao/CS6207/dataset/']
binary_dir = '/home/qihao/CS6207/binary'
words_dir = '/home/qihao/CS6207/binary/words'
event2word_dict, word2event_dict = pickle.load(open(f"{binary_dir}/music_dict.pkl", 'rb'))
event2word_dict['Prosody'];

In [181]:
for split in ['train', 'valid', 'test']:
    data2binary(dataset_dirs=dataset_dirs,
                words_dir=words_dir,
                split=split,
                word2event_dict=word2event_dict,
                event2word_dict=event2word_dict)

100%|████████████████████████████████████████| 473/473 [00:03<00:00, 131.37it/s]


| # train_tokens: 222227; train_items: 397


100%|██████████████████████████████████████████| 59/59 [00:00<00:00, 131.06it/s]

| # valid_tokens: 27593; valid_items: 49



100%|██████████████████████████████████████████| 60/60 [00:00<00:00, 150.11it/s]


| # test_tokens: 28047; test_items: 53


In [ ]:
from miditoolkit.midi.containers import Marker, Instrument, TempoChange, Note
def write(words, output_dir, midi_name, word2event):
    notes_all = []
    markers = []
    bar_cnt = -1
    positions = 0
    midi_obj = miditoolkit.midi.parser.MidiFile()
    event_type_list = []
    notes_all = []

    for event in words:
        bar_id, pos_id, pitch_id, dur_id, phrase_id = event[0], event[1], event[2], event[3], event[4]
        
        bar = word2event['Bar'][bar_id]
        pos = word2event['Pos'][pos_id]
        pitch = word2event['Pitch'][pitch_id]
        dur = word2event['Dur'][dur_id]
        phrase = word2event['Phrase'][phrase_id]
        
        # print(f"{bar}, {pos}, {pitch}, {dur}, {phrase}")
        
        if ("Bar_" not in bar) or ("Pos_" not in pos) or ("Pitch_" not in pitch) or ("Dur_" not in dur) or (("<true>" not in phrase) and ("<false>" not in phrase)):
            continue
        bar_num = int(bar.split('_')[1])
        pos_num = int(pos.split('_')[1])
        pitch_num = int(pitch.split('_')[1])
        dur_num = int(dur.split('_')[1])
        phrase_bool = True if phrase == '<true>' else False
        
        start = bar_num*1920 + pos_num
        end = start + dur_num
        notes_all.append(
            Note(pitch=pitch_num, start=start, end=end, velocity=80)
        )
        if phrase_bool:
            markers.append(Marker(time=start, text='Phrase'))
        
    # tempo
    midi_obj.tempo_changes.append(
                TempoChange(tempo=65, time=0))

    # marker
    midi_obj.markers.extend(markers)

    # track
    piano_track = Instrument(0, is_drum=False, name='melody')

    # notes
    piano_track.notes = notes_all
    midi_obj.instruments = [piano_track]

    # save
    tgt_melody_pth = os.path.join(output_dir, f"{midi_name.strip()}.mid")
    
    midi_obj.dump(tgt_melody_pth)

    return output_dir

Traceback (most recent call last):
  File "/tmp/ipykernel_2287134/3748347528.py", line 3, in data_to_binary
    src_words, tgt_words = tokenise(midi_pth, event2word_dict)
  File "/tmp/ipykernel_2287134/1822061069.py", line 52, in tokenise
    'strength':event2word_dict['Prosody'][f'<sent_{src_sent_id}>'],
KeyError: '<sent_72>'
Traceback (most recent call last):
  File "/tmp/ipykernel_2287134/3748347528.py", line 3, in data_to_binary
    src_words, tgt_words = tokenise(midi_pth, event2word_dict)
  File "/tmp/ipykernel_2287134/1822061069.py", line 52, in tokenise
    'strength':event2word_dict['Prosody'][f'<sent_{src_sent_id}>'],
KeyError: '<sent_72>'
Traceback (most recent call last):
  File "/tmp/ipykernel_2287134/3748347528.py", line 3, in data_to_binary
    src_words, tgt_words = tokenise(midi_pth, event2word_dict)
  File "/tmp/ipykernel_2287134/1822061069.py", line 52, in tokenise
    'strength':event2word_dict['Prosody'][f'<sent_{src_sent_id}>'],
KeyError: '<sent_72>'
Traceback (mo

In [152]:
def debinarise (data_item, word2event_dict):
    out_words = []
    for tgt_word in data_item['tgt_words']:
        out_words.append((
            tgt_word['bar'], tgt_word['pos'], tgt_word['token'], tgt_word['dur'], tgt_word['phrase']
        ))
    write(words=out_words, 
          output_dir='/home/qihao/CS6207/debinarise', 
          midi_name=data_item['item_name'], 
          word2event=word2event_dict)

In [155]:
dataset_test = np.load("/home/qihao/CS6207/binary/words/test_words.npy", allow_pickle=True)
print(len(dataset_test))
for idx, data_item in enumerate(dataset_test):
    if idx > 1:
        break
    debinarise(data_item, word2event_dict)

388
/home/qihao/CS6207/debinarise/月满西楼_seg0_1_Seg2.mid.mid
/home/qihao/CS6207/debinarise/wikifonia_3009_seg0_1_Seg1.mid.mid
